<div style='color: white; background-color: red;font-size: 33px;margin: 5px auto;border-radius: 5px;'> 
   <h1 style='text-align: center;'>Applied Data Science Capstone</h1>
</div>

<div style='color: white; background-color: lime;font-size: 32px;margin: 5px auto;border-radius: 5px;'> 
   <h2 style='text-align: center;'> Week 3 </h2>
   <h3 style='text-align: center;'>Segmenting and Clustering Neighborhoods in Toronto</h3>
</div>

In [64]:
#import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import requests
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

# Table of  Neighborhoods in Toronto
## Extracting Data: from wikipedia page and create table with pandas DataFrame.

### open url page and get html doc,then extract table what class 'wikitable sortable' by BeautifulSoup 

In [65]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_='wikitable sortable')

### extract columns name and data from table by using 'findAll' function, then create dataframe neighborhoods_df.

In [66]:
cols = []
data = []
for th in table.findAll('th'):
    col = th.getText(strip=True)
    cols.append(col)
for tr in table.findAll('tr'):
    if tr.findAll('th') == []:
        data.append([td.getText(strip=True) for td in tr.findAll('td')])
neighborhoods_df = pd.DataFrame(data, columns=cols)
neighborhoods_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Cleaning Data

### drop rows where 'Borough' feature is 'Not assigned' and replace where 'Neighbourhood' feature is 'Not assigned'.

In [67]:
drop_indexs = neighborhoods_df[neighborhoods_df['Borough'] == 'Not assigned'].index
neighborhoods_df = neighborhoods_df.drop(drop_indexs, axis=0)
neighborhoods_df.reset_index(drop=True, inplace=True)

In [68]:
replace_indexs = neighborhoods_df[neighborhoods_df['Neighbourhood'] == 'Not assigned'].index
neighborhoods_df.iloc[replace_indexs, 2] = neighborhoods_df['Borough'][replace_indexs]

### merge neighborhoods with same postcode into single cell

In [69]:
neighborhoods_df = neighborhoods_df.groupby(['Postcode', 'Borough']).agg(lambda x: ', '.join(x)).reset_index()

### get  latitude and longitude using geocoder lib

In [70]:
latitude  = []
longitude = []
postcode  = neighborhoods_df['Postcode']
for code in postcode:
    Geo = geocoder.arcgis(f'{code}, Toronto, Ontario')
    coordinates = Geo.latlng
    latitude.append(coordinates[0])
    longitude.append(coordinates[1])
neighborhoods_df['Latitude']  = latitude
neighborhoods_df['Longitude'] = longitude

In [71]:
neighborhoods_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [72]:
neighborhoods_df.shape

(103, 5)

<div style='color: white; background-color: lime;font-size: 32px;margin: 5px auto;border-radius: 5px;'> 
   <h2 style='text-align: center;'> Week 4 </h2>
   <h3 style='text-align: center;'>Capstone Project - The Battle of Neighborhoods (Week 1)</h3>
</div>